In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
# Activate mypy type checking and black formatting
%load_ext nb_black
%load_ext nb_mypy

In [ ]:
import nbimporter  # Activate import functionality for notebooks

# Chess-AI Statistik

Dieses Notebook lässt verschiedene AI-Versionen gegeneinander spielen und fasst die Ergebnisse in einer CSV-Datei zusammen.

Alle Optionen für die Ausführung können über die Umgebungsvariablen gesetzt werden. Das Notebook kann auch unbeaufsichtigt über [GitHub Actions](https://github.com/piuswalter/Chess-AI/actions/workflows/statistics.yml) ausgeführt werden. Der Workflow kann dort auf jedem existierenden Branch manuell gestartet werden. Der gewählte Branch entscheidet über den verwendeten Code. Ein Durchlauf mit den Standardoptionen (`development`-Branch) dauert derzeit etwa 1 Stunde. Die maximal erlaubte Laufzeit mit GitHub Actions beträgt 6 Stunden.

Verfügbare Versionen:
- `AI-Base-Class.ipynb`: Die abstrakte Basisklasse für alle AI-Versionen, welche das Eröffnungs- und Endspiel implementiert.
- `Exercise01AI`: Die erste Version der AI, welche im Mittelspiel zufällige Züge auswählt.
- `Exercise02AI`: Die zweite Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und dem einfachen Materialwert auswählt.
- `Exercise03AI`: Die dritte Version der AI, welche im Mittelspiel Züge mithilfe des Minimax-Algorithmus und der `simple evaluation function` auswählt.
- `Exercise04AI`: Die vierte Version der AI, welche im Vergleich zur dritten Version mithilfe von Memoisierung beschleunigt wurde.
- `Exercise05AI`: Die fünfte Version der AI, welche die dritte Version um das Alpha-Beta-Pruning erweitert.
- `Exercise06AI`: Die sechste Version der AI, welche die fünfte Version erneut um Memoisierung erweitert.
- `Exercise07AI`: Die siebte Version der AI, welche die sechste Version um Progressive Deepening erweitert.
- `Exercise08AI`: Die achte Version der AI, welche die siebte Version um die Singular Value Extension erweitert.
- `StockfishPlayer`: Ein Spieler, welcher durch die Stockfish-Engine gesteuert wird.

## Optionen manuell anpassen

Die folgende Zelle kann dazu genutzt werden um die Optionen manuell anzupassen.

In [ ]:
import json

manual_config = json.loads(
    """
{
  "players": [
    {
      "Exercise01AI": {}
    },
    {
      "Exercise02AI": {
        "search_depth": 3
      }
    },
    {
      "Exercise03AI": {
        "search_depth": 3
      }
    },
    {
      "Exercise04AI": {
        "search_depth": 3
      }
    },
    {
      "Exercise05AI": {
        "search_depth": 4
      }
    },
    {
      "Exercise06AI": {
        "search_depth": 4
      }
    },
    {
      "Exercise07AI": {
        "search_depth": 4
      }
    },
    {
      "Exercise08AI": {
        "search_depth": 4,
        "max_depth": 8
      }
    },
    {
      "StockfishPlayer": {
        "elo": 1500,
        "time_limit": 0.1
      }
    }
  ],
  "opponents": [
    {
      "Exercise01AI": {}
    },
    {
      "Exercise02AI": {
        "search_depth": 3
      }
    },
    {
      "Exercise03AI": {
        "search_depth": 3
      }
    },
    {
      "Exercise04AI": {
        "search_depth": 3
      }
    },
    {
      "Exercise05AI": {
        "search_depth": 4
      }
    },
    {
      "Exercise06AI": {
        "search_depth": 4
      }
    },
    {
      "Exercise07AI": {
        "search_depth": 4
      }
    },
    {
      "Exercise08AI": {
        "search_depth": 4,
        "max_depth": 8
      }
    },
    {
      "StockfishPlayer": {
        "elo": 1500,
        "time_limit": 0.1
      }
    }
  ],
  "seed": 0,
  "repetitions": 5,
  "concurrent_games": 1,
  "exclude_reflexive_pairs": false,
  "make_symmetric_pairs": true
}
"""
)

In [ ]:
import os

# Import all versions
from AIBaseClass import ChessAI
from Exercise01AI import Exercise01AI
from Exercise02AI import Exercise02AI
from Exercise03AI import Exercise03AI
from Exercise04AI import Exercise04AI
from Exercise05AI import Exercise05AI
from Exercise06AI import Exercise06AI
from Exercise07AI import Exercise07AI
from Exercise08AI import Exercise08AI
from StockfishPlayer import StockfishPlayer
from Main import run_games

available_versions = {
    "Exercise01AI": Exercise01AI,
    "Exercise02AI": Exercise02AI,
    "Exercise03AI": Exercise03AI,
    "Exercise04AI": Exercise04AI,
    "Exercise05AI": Exercise05AI,
    "Exercise06AI": Exercise06AI,
    "Exercise07AI": Exercise07AI,
    "Exercise08AI": Exercise08AI,
    "StockfishPlayer": StockfishPlayer,
}

Die Methode `get_config` lädt eine gegebene Konfiguration aus den Umgebungsvariablen und gibt diese zurück.

In [ ]:
def get_config() -> dict:
    env_config = os.environ.get("config")
    if env_config:
        return json.loads(env_config)
    return manual_config

Die `cleanup_game_folder`-Funktion löscht alle Dateien im `games`-Ordner. Dies verhindert, dass bereits existierende Dateien ungewollt in den Ergebnissen enthalten sind.

In [ ]:
import os


def cleanup_game_folder() -> None:
    """Deletes all files inside the games folder."""
    for filename in os.listdir("../games"):
        if filename.endswith(".pgn") or filename.endswith(".csv"):
            os.remove(f"../games/{filename}")

Die Funktion `create_player_config` bekommt eine Spielerklasse, einen Spielernamen sowie die Konfiguration für diesen Spieler und gibt die Spielerklasse zusammen mit ihrer Konfiguration zurück. Die Konfiguration ist dabei `immutable` damit sie in ein `set` hinzugefügt werden kann.

In [ ]:
def create_player_config(player: type, name: str, config: dict) -> tuple[type, tuple]:
    """Returns a player class with the contructor arguments as immutable objects."""
    return player, tuple({"player_name": name, **config}.items())

Die Methode `build_pairs` bekommt zwei Listen von ChessAI-Klassen und eine Konfiguration als Dictionary. Sie bildet das kartesische Produkt der beiden Listen, wobei jeweils die Spieler-Konfiguration hinzugefügt wird.
Optional können dabei reflexive Paare (Spieler = Gegner) vermieden werden und symmetrische Paare (Farbenwechsel) erzeugt werden.

In [ ]:
def build_pairs(
    players: list[type], opponents: list[type], config: dict
) -> set[tuple[tuple[type, tuple], tuple[type, tuple]]]:
    """Builds cartesian pairs of two iterables"""
    make_symmetric_pairs = config["make_symmetric_pairs"]
    exclude_reflexive_pairs = config["exclude_reflexive_pairs"]
    result: set[tuple[ChessAI, ChessAI]] = set()
    for i, player in enumerate(players):
        for j, opponent in enumerate(opponents):
            if exclude_reflexive_pairs and player == opponent:
                continue
            player_1_config = config["players"][i][player.__name__]
            player_2_config = config["opponents"][j][opponent.__name__]
            player_1 = create_player_config(player, "White", player_1_config)
            player_2 = create_player_config(opponent, "Black", player_2_config)
            result.add((player_1, player_2))
            if make_symmetric_pairs and (
                not player in opponents or not player_1_config == player_2_config
            ):
                s_player_1 = create_player_config(opponent, "White", player_2_config)
                s_player_2 = create_player_config(player, "Black", player_1_config)
                result.add((s_player_1, s_player_2))
    return result

Die `build_player_pairs`-Funktion ermittelt die angegebenen Spieler und Gegner, bildet eine Liste von Spielerpaaren und gibt diese nach dem Klassennamen sortiert zurück.


In [ ]:
def build_player_pairs(
    config: dict,
) -> list[tuple[tuple[type, tuple], tuple[type, tuple]]]:
    """Builds the sorted cartesian product of players x opponents and opponents x players"""
    # Get players and opponents
    players = [
        available_versions[tuple(player.keys())[0]] for player in config["players"]
    ]
    opponents = [
        available_versions[tuple(opponent.keys())[0]]
        for opponent in config["opponents"]
    ]
    # Build and return pairs
    player_pairs = build_pairs(players, opponents, config)
    return sorted(player_pairs, key=lambda x: (x[0][0].__name__, x[1][0].__name__))

Die `run`-Funktion nimmt ein Spielerpaar, eine Paarnummer, die Anzahl aller Paare sowie einen Seed und führt ein Spiel mit dem gewünschten Seed durch.

In [ ]:
import multiprocessing


def run(
    player_pair: tuple[tuple, tuple], pair_num: int, total_pairs: int, seed: int
) -> None:
    """Runs a game with the given player pair."""
    prefix = f"[{str(multiprocessing.current_process().pid)}]: "
    player_1, config_white = player_pair[0]
    player_2, config_black = player_pair[1]
    white_player = player_1(**dict(config_white))
    black_player = player_2(**dict(config_black))
    print(prefix + f"Starting game for player pair {pair_num + 1} of {total_pairs}")
    try:
        run_games(white_player, black_player, 1, seed)
    except Exception as e:
        raise RuntimeError(
            f"{prefix} Error in game '{player_1.__name__}' (white) against '{player_2.__name__}' (black) with seed '{seed}'!"
        ) from e

Die `run_statistics`-Funktion nimmt eine Liste von Spielerpaaren und führt für jedes Paar die gewünschte Anzahl an Spielen mit dem gewünschten Seed durch. Falls angegeben werden die gewünschte Anzahl an Spielen parallel in mehreren Prozessen ausgeführt.

In [ ]:
from itertools import repeat


def run_statistics(player_pairs: list[tuple[ChessAI, ChessAI]], config: dict) -> None:
    """Runs one or more games with each player pair."""
    process_count = config["concurrent_games"]
    is_human_run = not bool(os.environ.get("CI", False))
    original_seed = config["seed"]
    repetitions = config["repetitions"]
    total_pairs = len(player_pairs)
    print(f"Total games: {total_pairs * repetitions}")
    for game_round in range(repetitions):
        seed = original_seed + game_round
        arguments = [
            (pair, i, total_pairs, seed) for i, pair in enumerate(player_pairs)
        ]
        print(
            f"Starting game round {game_round + 1} of {repetitions} using seed {seed}"
        )
        # Let all player pairs play
        if is_human_run:
            for args in arguments:
                run(*args)
        else:
            with multiprocessing.Pool(process_count) as executor:
                list(executor.starmap(run, arguments))

Die `generate_stats`-Funktion lädt die, als Kommentar gespeicherten, Statistiken für jeden Zug und bereitet sie für den CSV-Export auf.

In [ ]:
import chess.pgn as pgn


def generate_stats(nodes: pgn.Mainline[pgn.ChildNode]) -> dict:
    """Collects and generates stats from .pgn file comments"""
    # Prepare data
    stats = {}
    raw_stats: list[dict] = [json.loads("{" + node.comment + "}") for node in nodes]
    raw_stats_white = [raw_stats[i] for i in range(0, len(raw_stats), 2)]
    raw_stats_black = [raw_stats[i] for i in range(1, len(raw_stats), 2)]
    # Get stockfish elo and time limit
    for node in raw_stats:
        if "elo" in node:
            stats["elo"] = node["elo"]
            stats["time_limit"] = node["time_limit"]
            break
    # Get last game state
    stats["last_state"] = raw_stats[-2]["state"].split(".")[1]
    # Get total calculation time
    engine_time_white = sum(node["time"] for node in raw_stats_white)
    engine_time_black = sum(node["time"] for node in raw_stats_black)
    # Get average move calculation time
    stats["move_time_white"] = engine_time_white / len(raw_stats_white)
    stats["move_time_black"] = engine_time_black / len(raw_stats_black)
    # Get average depth of all moves (if applicable)
    depth_list_white = [
        node["avg_depth"] for node in raw_stats_white if "avg_depth" in node
    ]
    depth_list_black = [
        node["avg_depth"] for node in raw_stats_black if "avg_depth" in node
    ]
    stats["average_depth_white"] = sum(depth_list_white) / (len(depth_list_white) or 1)
    stats["average_depth_black"] = sum(depth_list_black) / (len(depth_list_black) or 1)
    # Get max depth of all moves (if applicable)
    max_depth_list_black = [
        node["max_depth"] for node in raw_stats_black if "max_depth" in node
    ]
    max_depth_list_white = [
        node["max_depth"] for node in raw_stats_white if "max_depth" in node
    ]
    stats["max_depth_white"] = (
        max(max_depth_list_white) if max_depth_list_white else "-"
    )
    stats["max_depth_black"] = (
        max(max_depth_list_black) if max_depth_list_black else "-"
    )
    # Get the percentage of overall cache hits
    cache_tries_white = sum(
        node["cache_tries"] for node in raw_stats_white if "cache_tries" in node
    )
    cache_tries_black = sum(
        node["cache_tries"] for node in raw_stats_black if "cache_tries" in node
    )
    cache_hits_white = sum(
        node["cache_hits"] for node in raw_stats_white if "cache_hits" in node
    )
    cache_hits_black = sum(
        node["cache_hits"] for node in raw_stats_black if "cache_hits" in node
    )
    stats["cache_hits_white"] = (
        cache_hits_white / cache_tries_white if cache_tries_white else "-"
    )
    stats["cache_hits_black"] = (
        cache_hits_black / cache_tries_black if cache_tries_black else "-"
    )
    # Get the maximum cache size (if applicable)
    cache_list_white = [
        node["cache_size_mb"] for node in raw_stats_white if "cache_size_mb" in node
    ]
    cache_list_black = [
        node["cache_size_mb"] for node in raw_stats_black if "cache_size_mb" in node
    ]
    stats["max_cache_size_mb_white"] = (
        max(cache_list_white) if cache_list_white else "-"
    )
    stats["max_cache_size_mb_black"] = (
        max(cache_list_black) if cache_list_black else "-"
    )
    # Get the complete list of moves
    stats["move_list"] = ",".join(entry["move"] for entry in raw_stats)
    return stats

Die `collect_results`-Funktion importiert die Ergebnisse aus allen `.pgn`-Dateien im `games`-Ordner und bereitet sie als Dictionary auf.

In [ ]:
# Collect results from .pgn files


def collect_results(folder: str) -> list[dict]:
    """Scans all .pgn files inside the games folder and collects the game results as dictionary."""
    # All results are presented from the view of the white color
    results_map_white = {"1-0": "Won", "0-1": "Lost", "1/2-1/2": "Remis", "*": "Remis"}
    results_map_black = {"0-1": "Won", "1-0": "Lost", "1/2-1/2": "Remis", "*": "Remis"}
    games: list[dict] = []

    # Iterate over all .pgn files
    for filename in os.listdir(folder):
        if not filename.endswith(".pgn"):
            continue
        with open(os.path.join(folder, filename)) as pgn_file:
            pgn_game = pgn.read_game(pgn_file)
        if not pgn_game:
            continue

        # Collect results
        results = {
            "round": pgn_game.headers["Round"],
            "player": pgn_game.headers["White"],
            "opponent": pgn_game.headers["Black"],
            "result_white": results_map_white[pgn_game.headers["Result"]],
            "result_black": results_map_black[pgn_game.headers["Result"]],
            "outcome": pgn_game.headers["Outcome"],
            "duration": float(pgn_game.headers["Duration"]),
            "seed": pgn_game.headers["Seed"],
            "commit": pgn_game.headers["Commit"],
            "depth_white": pgn_game.headers["Depth"].split("-")[0],
            "depth_black": pgn_game.headers["Depth"].split("-")[1],
            "total_moves": pgn_game.end().ply(),
            "filename": filename,
            **generate_stats(pgn_game.mainline()),
        }

        # Save results
        games.append(results)
    return games

Die `export_to_csv`-Funktion exportiert eine gegebene Sammlung (Dictionary) von Spielergebnissen als CSV-Datei.

In [ ]:
import csv
from datetime import datetime


def export_to_csv(games: list[dict], target_folder: str) -> None:
    """Exports the given results dictionary to csv."""
    filename = f'statistics_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")}.csv'
    path = os.path.join(target_folder, filename)
    with open(path, "w", newline="") as csv_file:
        csv_writer = csv.writer(
            csv_file, delimiter=";", quotechar='"', quoting=csv.QUOTE_MINIMAL
        )
        # Write headlines
        csv_writer.writerow(
            [
                "Player",
                "Result white",
                "Depth white",
                "Average depth white",
                "Max depth white",
                "Average move time white (s)",
                "Cache hits white (%)",
                "Max cache size white (mb)",
                "Opponent",
                "Result black",
                "Depth black",
                "Average depth black",
                "Max depth black",
                "Average move time black (s)",
                "Cache hits black (%)",
                "Max cache size black (mb)",
                "Outcome",
                "Moves",
                "Duration (s)",
                "Last state",
                "Round",
                "Seed",
                "Commit Hash",
                "File",
                "Move list (uci)",
                "Comment",
            ]
        )
        # Write results
        for game in sorted(games, key=lambda game: game["filename"]):
            for key in game:
                if isinstance(game[key], float):
                    game[key] = str(round(game[key], 2)).replace(".", ",")
            csv_writer.writerow(
                [
                    game["player"],
                    game["result_white"],
                    game["depth_white"],
                    game["average_depth_white"],
                    game["max_depth_white"],
                    game["move_time_white"],
                    game["cache_hits_white"],
                    game["max_cache_size_mb_white"],
                    game["opponent"],
                    game["result_black"],
                    game["depth_black"],
                    game["average_depth_black"],
                    game["max_depth_black"],
                    game["move_time_black"],
                    game["cache_hits_black"],
                    game["max_cache_size_mb_black"],
                    game["outcome"].split(".")[1],
                    game["total_moves"],
                    game["duration"],
                    game["last_state"],
                    game["round"].replace("/", " of "),
                    game["seed"],
                    game["commit"],
                    game["filename"],
                    game["move_list"],
                    ""
                    if "elo" not in game
                    else f'elo: {game["elo"]}, time_limit: {game["time_limit"]}',
                ]
            )
    print(f"Exported {len(games)} games to '{path}'")

Die folgende Zelle bereinigt den `games`-Ordner, erstellt die Spielerpaare, startet die Spiele, sammelt die Ergebnisse und exportiert sie als CSV-Datei.

In [ ]:
# Run and export games
if __name__ == "__main__":
    # cleanup_game_folder()
    config = get_config()
    player_pairs = build_player_pairs(config)
    run_statistics(player_pairs, config)
    results = collect_results("../games")
    export_to_csv(results, "../games")